<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_question_KcBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화 중 질문을 선별하는 코드(로컬)

In [ ]:
!pip install soynlp
!pip install datasets
!pip install kss
!pip install accelerate -U
import accelerate
import random
import pandas as pd
import numpy as np
import re
import os
import tensorflow as tf
import urllib.request
import torch
import torch.nn as nn
from kss import Kss
from transformers.modeling_outputs import SequenceClassifierOutput
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from tqdm import tqdm
from torch.nn.functional import cross_entropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, Trainer, TrainingArguments, RobertaConfig, shape_list, BertTokenizer, TFBertModel, RobertaModel, RobertaTokenizer, TrainerCallback, pipeline, BertForMaskedLM, BertConfig, BertForSequenceClassification
from datasets import Dataset, load_dataset, ClassLabel
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

In [ ]:
#Sequence classification으로 훈련된 모델 로드
finetuned_model_load_path = '/content/drive/My Drive/Finetuned_Model_judge_question'

finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_load_path)
finetuned_model = BertForSequenceClassification.from_pretrained(finetuned_model_load_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#카카오톡 대화내용 불러오고 데이터프레임으로 받기
file_path = '/content/drive/My Drive/talk_preprocess_result_short.xlsx'

df = pd.read_excel(file_path)

In [ ]:
# Trainer가 알아서 gpu를 감지하고 모델과 데이터를 gpu로 옮겨줌
#GPU 사용 코드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#파이프라인을 사용하는 경우 GPU로 모델을 옮겨준 뒤 파이프라인에 모델을 넣어줘야 함
finetuned_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(40596, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
#데이터 처리 Pipeline 설정, top_k를 1로 설정시 최대로 측정된 label만 출력됨.
#텍스트 처리해보니 시퀀스 길이가 539인 행 존재 -> truncation 진행, 최대 길이는 512
text_classifier = TextClassificationPipeline(
    tokenizer=finetuned_tokenizer,
    model=finetuned_model,
    top_k = 1,
    truncation = True,
    batch_size = 128,
    device = device
)

In [ ]:
# #결측치 제거
# df['text'] = df['text'].dropna(axis = 0)

In [ ]:
# #결측치 검증
# print(type(text[:5]))
# print(all(isinstance(item, str) for item in text))
# for i, item in enumerate(text):
#   if not isinstance(item, str):
#     print(f"Index {i}: {item} (Type: {type(item)})")

In [ ]:
#결측치 제거
df = df.replace('None', np.nan).dropna(axis = 0)

#모델로 prediction_labels 리스트 생성.
text = df['text'].to_list()

#tqdm으로 진행률 표시
predictions= []
for i in tqdm(range(0, len(text), 128), desc='Classifying'):
    batch = text[i:i+128]
    predictions.extend(text_classifier(batch))

# predictions = text_classifier(text)
prediction_labels = ['question' if prediction[0]['label'] == 'LABEL_1' else 'not question' for prediction in predictions]


#Dataframe에 새 열로 추가.
df['label'] = prediction_labels

Classifying: 100%|██████████| 62/62 [01:14<00:00,  1.21s/it]


In [ ]:
print(df)
print(df[df['label'] == 'question'])

In [ ]:
#.csv 파일로 google drive에 저장
#illegalcharactererror에 의해 .xlsx 대신 .csv로 저장
save_path = '/content/drive/My Drive/judge_question_result_short_KcBERT.csv'

df.to_csv(save_path)